<a href="https://colab.research.google.com/github/annguyenhuynh/Anhuynh/blob/main/Spark_Chap25_FeatureManipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=1bf4f8d7c8c7b093a8dfd66be2aee717c332dea1aa98063b01743d5042995568
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=487f8fa2ea9d5bae12ba2dcfb391d3faca0411d0a8a723eb30ea70c6a69fa36a
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
spark=SparkSession.builder.getOrCreate()

In [5]:
sales = spark.read.format("csv")\
  .option("header",True)\
  .option("inferSchema",True)\
  .load("/content/drive/MyDrive/bestModel/metadata/databricks Spark-The-Definitive-Guide master data-retail-data_by-day/*.csv")\
  .coalesce(5)\
  .where("Description is NOT NULL")

fakeInDF = spark.read.parquet("/content/drive/MyDrive/databricks Spark-The-Definitive-Guide master data-simple-ml-integers")

simpleDF = spark.read.json("/content/drive/MyDrive/databricks Spark-The-Definitive-Guide master data-simple-ml")

scaledDF = spark.read.parquet("/content/drive/MyDrive/databricks Spark-The-Definitive-Guide master data-simple-ml-scaling")

In [6]:
sales.cache()
sales.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    21544|SKULLS  WATER TRA...|      48|2011-12-05 08:38:00|     0.85|   14075.0|United Kingdom|
|   580538|    23126|FELTCRA

In [7]:
from pyspark.ml.feature import RFormula

supervised = RFormula(formula= "lab ~ . + color:value1 + color:value2")
supervised.fit(simpleDF).transform(simpleDF).show()

+-----+----+------+------------------+--------------------+-----+
|color| lab|value1|            value2|            features|label|
+-----+----+------+------------------+--------------------+-----+
|green|good|     1|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
| blue| bad|     8|14.386294994851129|(10,[2,3,6,9],[8....|  0.0|
| blue| bad|    12|14.386294994851129|(10,[2,3,6,9],[12...|  0.0|
|green|good|    15| 38.97187133755819|(10,[1,2,3,5,8],[...|  1.0|
|green|good|    12|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
|green| bad|    16|14.386294994851129|(10,[1,2,3,5,8],[...|  0.0|
|  red|good|    35|14.386294994851129|(10,[0,2,3,4,7],[...|  1.0|
|  red| bad|     1| 38.97187133755819|(10,[0,2,3,4,7],[...|  0.0|
|  red| bad|     2|14.386294994851129|(10,[0,2,3,4,7],[...|  0.0|
|  red| bad|    16|14.386294994851129|(10,[0,2,3,4,7],[...|  0.0|
|  red|good|    45| 38.97187133755819|(10,[0,2,3,4,7],[...|  1.0|
|green|good|     1|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
| blue| ba

In [8]:
from pyspark.ml.feature import StandardScaler

In [9]:
sScaler = StandardScaler().setInputCol("features")
sScaler.fit(scaledDF).transform(scaledDF).show()

+---+--------------+-----------------------------------+
| id|      features|StandardScaler_bdf85cc89f90__output|
+---+--------------+-----------------------------------+
|  0|[1.0,0.1,-1.0]|               [1.19522860933439...|
|  1| [2.0,1.1,1.0]|               [2.39045721866878...|
|  0|[1.0,0.1,-1.0]|               [1.19522860933439...|
|  1| [2.0,1.1,1.0]|               [2.39045721866878...|
|  1|[3.0,10.1,3.0]|               [3.58568582800318...|
+---+--------------+-----------------------------------+



# **PCA** - PRINCIPAL COMPONENTS ANALYSIS

*   A mathematical technique for finding the most imoortant aspects of our data (the principal components)
*   It changes the feature representation of our data by creating a new set of features("aspects"). Each new feature is a combination of the original features.
*   The power of PCA is that it can create a **SMALLER** set of more **MEANINGFUL** feaures to be input of our mode, at the potential cost of our interpretability.
*   We want to use PCA when we have large input dataset and want to reduce the total number of features. This frequently comes up in text analysis where the entire feature space is massive and many of the features are largely irrelevant.
*   Using PCA, we can find the most important combinations if features and only include those in our ML models.







In [ ]:
from pyspark.ml.feature import PCA

In [ ]:
pca = PCA().setInputCol('features').setK(2)
pca.fit(scaledDF).transform(scaledDF).show(40,False)

+---+--------------+------------------------------------------+
|id |features      |PCA_95df978c2243__output                  |
+---+--------------+------------------------------------------+
|0  |[1.0,0.1,-1.0]|[0.0713719499248417,-0.45266548881478363] |
|1  |[2.0,1.1,1.0] |[-1.680494698407372,1.259340132221916]    |
|0  |[1.0,0.1,-1.0]|[0.0713719499248417,-0.45266548881478363] |
|1  |[2.0,1.1,1.0] |[-1.680494698407372,1.259340132221916]    |
|1  |[3.0,10.1,3.0]|[-10.872398139848944,0.030962697060152422]|
+---+--------------+------------------------------------------+



# Preserving the Variance
*   Select the hyperplane that preserves the **maximum** amount of varianace, as it will most likely lose less information than other projections
*   If you can plot them on a 2D hyperplane, you should choose the axis that **minimized the mean-squared distance** between the original dataset and its projection on that axis.



# Principal Components

*   PCA identifies the axis that accounts for the largest amount of variance

*   It also finds the 2nd axis, orthogonal to the first one, that accounts for the largest amount of remaining variance

*   The ith axis is called the ith *princial component (PC)** of the data

*   For each PC, PCA finds a zero-centered unit vector pointing in the direction of the PC. Since 2 opposing unit vectors lie on the same axis, the direction of the unit vectors returned by PCA is not stable.

*   To find PCs, use a **standard matrix factorization** called ***Singular Value Decomposition (SVD)*** that can decompose the training set matrix X into the matrix multiplication of 3 matrices
U S Vt, where V contains the unit vectors that define all the principal components that we are looking for

*   Before implemeting PCA, ensure that data is centered around the origin (zero-centered) by performing data normalizing and scaling.
    
                  *X_centered = X-X.mean(axis=0)*
                  *U,S,Vt = np.linagl.svd(X_centered)*

*  In Python, use **sklearn.decompostion.PCA** and set *n_components* to the number of dimensions you want to reduce to.







In [ ]:
scaledDF.show(10,False)

+---+--------------+
|id |features      |
+---+--------------+
|0  |[1.0,0.1,-1.0]|
|1  |[2.0,1.1,1.0] |
|0  |[1.0,0.1,-1.0]|
|1  |[2.0,1.1,1.0] |
|1  |[3.0,10.1,3.0]|
+---+--------------+



In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=0.95)
scaled_data = np.array(scaledDF.select("features").collect())
scaled_data = scaled_data.reshape(scaled_data.shape[0],-1)
X_reduced = pca.fit_transform(scaled_data)

In [ ]:
X_reduced

array([[-2.88950068],
       [-1.13763403],
       [-2.88950068],
       [-1.13763403],
       [ 8.05426941]])

*   In the above code, instead of specifying the number of principal components to be reserved, we set n_components to a float between 0.0 and 1.0, indicating the ratio of variance you wish to preserve.






# Math intuition behind PCA


*   The goal of PCA is to find the unit vector that captures the max variance.

*   The technique to select the best unit vector is called EigenDecomposition. Eigendecomposition breaks down a matrix into its eigenvalues and eigenvectors to help you find the non-obvious and universal properties.

*   In linear algebra, an eigenvector, or characteristic vector is a vector that has its direction unchanged by a given linear transformation. More precisely, an eigenvector,
v of a linear transformation, T is scaled by a constant factor, λ when the linear transformation is applied to it:  **Tv = λv**. It is often important to know these vectors in linear algebra. The corresponding eigenvalue, characteristic value, or characteristic root is the multiplying factor
λ. (Wikipedia)

*   Geometrically, vectors are multi-dimensional quantities with magnitude and direction, often pictured as arrows. A linear transformation rotates, stretches, or shears the vectors upon which it acts. Its eigenvectors are those vectors that are only stretched, with neither rotation nor shear. The corresponding eigenvalue is the factor by which an eigenvector is stretched or squished. If the eigenvalue is negative, the eigenvector's direction is reversed (Wikipedia)



# Interaction

*   In some cases, you might know that a certain interaction between 2 variables is an important varialble to include in a downstream estimator.

*   RFormula: (y ~ x1+x2+x1:x2)








# Polynomial Expansion

*   Used to generate interaction variables of all the input columns. With polynomial expansions, we specify to what degree we would like to see various interactions.

*   If we have 2 input features, we'll get 4 (2x2) output features if we use 2nd degree polynomial. If we have 3 input features, we'll get 9 (3x3) output features; if we use 3rd degree polynomial, we will get 27(3*3*3) and so on.


In [10]:
from pyspark.ml.feature import PolynomialExpansion

In [11]:
pe = PolynomialExpansion().setInputCol("features").setDegree(2)
pe.transform(scaledDF).show()

+---+--------------+----------------------------------------+
| id|      features|PolynomialExpansion_4baf03b3fb6d__output|
+---+--------------+----------------------------------------+
|  0|[1.0,0.1,-1.0]|                    [1.0,1.0,0.1,0.1,...|
|  1| [2.0,1.1,1.0]|                    [2.0,4.0,1.1,2.2,...|
|  0|[1.0,0.1,-1.0]|                    [1.0,1.0,0.1,0.1,...|
|  1| [2.0,1.1,1.0]|                    [2.0,4.0,1.1,2.2,...|
|  1|[3.0,10.1,3.0]|                    [3.0,9.0,10.1,30....|
+---+--------------+----------------------------------------+



In [15]:
from pyspark.ml.feature import ChiSqSelector, Tokenizer

In [18]:
from pyspark.sql.functions import *

In [20]:
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn\
  .transform(sales.select("Description","CustomerId"))\
  .where("CustomerId is NOT NULL")
prechi = fittedCV.transform(tokenized)\
  .where("CustomerId is NOT NULL")
chisq = ChiSqSelector()\
  .setFeaturesCol("countVec")\
  .setLabelCol("CustomerId")\
  .setNumTopFeatures(2)
chisq.fit(prechi).transform(prechi)\
  .drop("CustomerId", "Description", "DescOut").show()


NameError: name 'fittedCV' is not defined

# ChiSqSelector

*   ChiSqSelector leverages statistical test to identify features that are not independent from the label we are trying to predict, and drop the uncorrelated features.

*   It's often used with categorical data to reduce the number of features you will input into your model, as well as to reduce the dimensionality of text data (in the form of frequencies or counts)

*   We can use **numTopFeatures**, which is ordered by p-values
*   or **percentile**, which takes a proportion of the input features (instead of just the top N values)
*   and **fpr** which sets the cutoff p-values






